# Benchmarking Circuit Depth for PEC on Real Hardware
***

## Imports

In [ ]:
import functools
import os
import time
from typing import List

import matplotlib.pyplot as plt

import cirq
import networkx as nx
import numpy as np
import qiskit
import qiskit_aer
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke 
from qiskit.providers.fake_provider import GenericBackendV2
from mitiq import benchmarks, pec
from qiskit_ibm_runtime import SamplerV2, Batch

## Variables for experiment

In [ ]:
depths = np.arange(10,201,10)
trials = 10
n_qubits = 2
shots = 10000
num_samples = 100
circuit_type= "mirror" # "rb"

## Initialize Real QPU

In [ ]:
# from qiskit_ibm_runtime import QiskitRuntimeService
# service = QiskitRuntimeService(channel="ibm_quantum", token = "<Your Token Here>")
# noisy_backend = service.backend("ibm_sherbrooke")

## General setup
Define ideal backend and seed for circuit generation.
Options for verbose output no longer in use. (TODO: remove or streamline)

In [ ]:
ideal_backend = qiskit_aer.AerSimulator()
noisy_backend = FakeSherbrooke()
# Random seed for circuit generation
seed: int = 1

# Display verbose output.
verbose: bool = False
# Give queue updates every this many seconds when running on hardware device.
verbose_update_time: int = 30

## Virtual and physical qubit architecture
Get line of physical qubits from backend: Hard-coded in right now, will hopefully be automatic someday.
Translate physical to virtual qubits.
Create "computer" a directed coupling map.

(Move to utils)

In [ ]:
def get_phys_qubits(n_qubits):
    # Physical qubits with a chain-like connectivity.
    ibm_sherbrooke_ordering = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 17, 30, 29, 28, 27, 26, 25, 24, 
                               23, 22, 21, 20, 33, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 55, 68, 
                               67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 71, 77, 78, 79, 80, 81, 82, 83, 
                               84, 85, 86, 87, 93, 106, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96, 
                               109, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126] # Up to 87 qubits

    return ibm_sherbrooke_ordering[: n_qubits]


In [ ]:
# Maps physical qubits to virtual ones
def get_layout(n_qubits):
    phys_qubits = get_phys_qubits(n_qubits)
    virt_qubits =[]
    for qubit_i in range(0,n_qubits):
        virt_qubits.append(qubit_i)
    layout =  dict(zip(phys_qubits, virt_qubits))
        
    return layout #keys: physical, values: virtual

In [ ]:
def get_computer(n_qubits):
    layout = get_layout(n_qubits)
    phys_edges = noisy_backend.coupling_map.get_edges()
    virt_edges = []
    for edge_i in range(0, int(len(phys_edges)/2)):
        phys_edge = phys_edges[edge_i]
        try:
            virt_edge = (layout[phys_edge[0]],layout[phys_edge[1]])
        except:
            # print("skip: This edge is not included in our layout")
            meaningless=1
        virt_edges.append(virt_edge)
        
    # Make connectivity graph 
    computer = nx.DiGraph()
    computer.add_edges_from(virt_edges[:n_qubits - 1])

    return computer

## Create desired circuit
(Move to utils)

In [ ]:
def get_circuit(circuit_type:str, n_qubits:int, depth: int, seed: int) -> tuple[qiskit.QuantumCircuit, str]:
    """Create circuit."""
    
    if circuit_type == "rb":
        circuit = benchmarks.generate_rb_circuits(
            n_qubits=2, 
            num_cliffords=depth, 
            seed=seed, 
            return_type="qiskit"
            )[0]
        return circuit, "00"

    elif circuit_type == "mirror":
        # Make connectivity graph 
        computer = get_computer(n_qubits)
        circuit, correct_bitstring = benchmarks.generate_mirror_circuit(
                nlayers=depth,
                two_qubit_gate_prob=1.0,
                connectivity_graph=computer,
                two_qubit_gate_name="CNOT",
                seed=seed,
                return_type="qiskit",
            )
        # Reversed because Qiskit is wrong endian.
        return circuit, "".join(map(str, correct_bitstring[::-1]))
    
    elif circuit_type == "long cnot":
        circuit = qiskit.QuantumCircuit(n_qubits)
        circuit.x(0)
        for qubit_i in range(0, n_qubits-1):
            circuit.cx(qubit_i, qubit_i + 1)
        correct_bitstring= "1"*(n_qubits - 1)
        return circuit, correct_bitstring

    else:
        print("what")
        return


## Count gates
(Move to utils)

In [ ]:
def get_num_cnot_count(circuit: qiskit.QuantumCircuit) -> int:
    """Determine number of cnot gates in a given `Circuit` object."""

    return circuit.count_ops().get("cx")


def get_avg_cnot_count(circuits: list[qiskit.QuantumCircuit]) -> float:
    """Determine average number of cnot gates present in
    list of `QuantumCircuit` objects."""
    return np.average([c.count_ops().get("cx", 0) for c in circuits]) # Returns 0 instead of None


def get_oneq_count(circuit: qiskit.QuantumCircuit) -> int:
    return len(circuit) - get_num_cnot_count(circuit)

## Define executor to use with mitiq
(TODO: ensure that its actually batched)

In [ ]:
def execute(
    circuits: list[qiskit.QuantumCircuit],
    backend,
    shots: int,
    correct_bitstring: str,
    verbose: bool,
    ) -> List[float]:
    """Executes the input circuit(s) and returns ⟨A⟩, where A = |correct_bitstring⟩⟨correct_bitstring| for each circuit."""

    if not isinstance(circuits, list):
        circuits = [circuits]
    if verbose:
        # Calculate average number of CNOT gates per circuit.
        print(f"Executing {len(circuits)} circuit(s) on {backend}.")
        print(f"Average cnot count in circuits: {get_avg_cnot_count(circuits)}")

    # Store all circuits to run in list to be returned.
    to_run: list[qiskit.QuantumCircuit] = []

    for circuit in circuits:
        circuit_to_run = circuit.copy()
        circuit_to_run.measure_all()
        to_run.append(
            qiskit.transpile(
                circuit_to_run,
                backend=backend,
                initial_layout= get_phys_qubits(circuit.num_qubits),
                optimization_level=0,  # Otherwise RB circuits are simplified to empty circuits.
        ))

    if verbose:
        # Calculate average number of CNOT gates per compiled circuit.
        print(f"Average cnot count in compiled circuits: {get_avg_cnot_count(to_run)}")

    batch = Batch(backend= backend)
    sampler = SamplerV2(mode = batch)
    result = sampler.run(to_run, shots=shots).result()
    counts = result[0].data.meas.get_counts()
    batch.close()

    if len(circuits) == 1:
        return [counts.get(correct_bitstring, 0.0) / shots]
    return [
        counts.get(correct_bitstring, 0.0) / shots #for count in counts
    ]

## Determine noise model
Calculate CNOT error for backend
(TODO: with backend as argument, set CNOT error to either fixed or qubit-dependent, figure out if backend has CNOT natively and construct if not)
(Move to utils)

In [ ]:
def get_cnot_error(backend, edge: tuple[int, int] = None) -> float:

    # cnot_error_prob = 0.01

    #translate virtual edge back to physical edge by inverting dictionary (layout)
    layout = get_layout(n_qubits)
    inv_layout = dict((v, k) for k, v in layout.items())

    # Find physical edge corresponding to virtual edge
    phys_edge = (inv_layout[edge[0]],inv_layout[edge[1]])

    # Build CNOT out of native gates
    rz_error_q0 = noisy_backend.properties().gate_error("rz", qubits=phys_edge[0])
    sqrtx_error_q0 = noisy_backend.properties().gate_error("sx", qubits=phys_edge[0])
    rz_error_q1 = noisy_backend.properties().gate_error("rz", qubits=phys_edge[1])
    ecr_error = noisy_backend.properties().gate_error("ecr", qubits=[phys_edge[0],phys_edge[1]])
    x_error_q1 = noisy_backend.properties().gate_error("x", qubits=phys_edge[1])
    cnot_error_prob = 1 - (1-rz_error_q0)*(1-rz_error_q0)*(1-sqrtx_error_q0)*(1-rz_error_q0)*(1-rz_error_q0)*(1-ecr_error)*(1-rz_error_q1)*(1-x_error_q1)

    # print(f"cnot_error_prob for edge {edge}: {cnot_error_prob}")
    
    return cnot_error_prob #return error prob for phys edge corresponding to input virt edge


def get_cnot_representation(backend, edge: tuple[int, int]) -> pec.OperationRepresentation:
    cnot_circuit = cirq.Circuit(
        cirq.CNOT(
            cirq.NamedQubit(f"q_{str(edge[0])}"),
            cirq.NamedQubit(f"q_{str(edge[1])}"),
        )
    )
    rep_exact_prob = 1 - np.sqrt(1 - get_cnot_error(backend,edge))
    return pec.represent_operation_with_local_depolarizing_noise(
        cnot_circuit,
        noise_level=rep_exact_prob,
    )

def get_representations(backend, computer: nx.Graph) -> list[pec.OperationRepresentation]:
    return [get_cnot_representation(backend, edge) for edge in computer.edges]

## Execute with noisy and ideal backend, with PEC and without

In [ ]:
def run_experiment(depth, trials, n_qubits, shots, num_samples, circuit_type):
    computer = get_computer(n_qubits)

    true_values_at_depth = []
    noisy_values_at_depth = []
    pec_values_at_depth = []
    cnot_counts_at_depth = []
    oneq_counts_at_depth = []
    for trial in range(trials):
        # Local seed is calculated in this way to ensure that we don't get repeat values in loop.
        local_seed = int(10**6 * depth + 10**3 * seed + trial)

        circuit, correct_bitstring = get_circuit(circuit_type, n_qubits, depth, local_seed)
        (true_value,) = execute(
            circuit,
            ideal_backend,
            shots,
            correct_bitstring,
            verbose=verbose,
        )
        true_values_at_depth.append(true_value)

        (noisy_value,) = execute(
            circuit,
            noisy_backend,
            shots,
            correct_bitstring,
            verbose=verbose
        )
        noisy_values_at_depth.append(noisy_value)

        pec_executor = functools.partial(
            execute,
            backend=noisy_backend,
            shots=shots // num_samples,
            correct_bitstring=correct_bitstring,
            verbose=verbose,
        )

        pec_value = pec.execute_with_pec(
            circuit,
            pec_executor,
            representations=get_representations(noisy_backend,computer),
            num_samples=num_samples,
            random_state=local_seed,
        )
        
        pec_values_at_depth.append(pec_value)

        cnot_counts_at_depth.append(get_num_cnot_count(circuit))
        oneq_counts_at_depth.append(get_oneq_count(circuit))

    return true_values_at_depth, noisy_values_at_depth, pec_values_at_depth, cnot_counts_at_depth, oneq_counts_at_depth

## Usefulness test
(TODO: remove this/alter to callout when no longer in error bars?)
(Move to utils)

NOT IMPLEMENTED RN FOR REAL

In [ ]:
def still_useful(avg_true_value, avg_noisy_value, avg_pec_value, std_noisy_value, std_pec_value):

    pec_diff = np.abs(avg_true_value - avg_pec_value)
    noisy_diff = np.abs(avg_true_value - avg_noisy_value)

    # If PEC worse than raw 
    if (pec_diff > noisy_diff):
        reason = "Worse than raw"
        return False, reason
    
    else:
        reason = "none"
        return True, reason

In [ ]:
# BE VERY CAREFUL WITH THIS
# Just to remove the UserWarning for representations

import warnings
warnings.filterwarnings("ignore")

## Loop over depths, extract values

In [ ]:

def test_depths(depths, trials, n_qubits, shots, num_samples, circuit_type):
    cnot_counts = []
    oneq_counts = []

    true_values=[]
    noisy_values = []
    pec_values = []

    avg_true_values=[]
    avg_noisy_values = []
    avg_pec_values = []

    std_true_values = []
    std_noisy_values = []
    std_pec_values = []

    still_useful_counter = 0

    for depth_i in range(0,len(depths)):

        print("Status: On depth", depths[depth_i], "with backend", noisy_backend, end="\n\n")

        true_values_at_depth, noisy_values_at_depth, pec_values_at_depth, cnot_counts_at_depth, oneq_counts_at_depth = run_experiment(depths[depth_i], trials, n_qubits, shots, num_samples, circuit_type)
        
        # Store values
        true_values.append(true_values_at_depth)
        noisy_values.append(noisy_values_at_depth)
        pec_values.append(pec_values_at_depth)

        # Store averages
        avg_true_values.append(np.average(true_values_at_depth))
        avg_noisy_values.append(np.average(noisy_values_at_depth))
        avg_pec_values.append(np.average(pec_values_at_depth))
        
        # Store standard deviations
        std_true_values.append(np.std(true_values_at_depth, ddof=1))
        std_noisy_values.append(np.std(noisy_values_at_depth, ddof=1))
        std_pec_values.append(np.std(pec_values_at_depth, ddof=1))

        # Store gate counts
        cnot_counts.append(cnot_counts_at_depth)
        oneq_counts.append(oneq_counts_at_depth)

        #Check usefulness
        still_useful_at_depth, reason = still_useful(avg_true_values[depth_i], 
                        avg_noisy_values[depth_i], 
                        avg_pec_values[depth_i], 
                        std_noisy_values[depth_i], 
                        std_pec_values[depth_i])
        
        # # Count how many times in a row we're not useful, once we get to five (a trend, rather than a fluke bad spot) end it
        # if still_useful_at_depth==True:
        #     print("Reset the count")
        #     still_useful_counter = 0
        # elif still_useful_at_depth==False:
        #     print("Add count:", reason)
        #     still_useful_counter += 1 
        # if still_useful_at_depth==False and still_useful_counter>=5:
        #     print("PEC is no longer useful after depth = ", depths[depth_i-1])
        #     # break
    return cnot_counts, oneq_counts, true_values, noisy_values, pec_values, avg_true_values, avg_noisy_values, avg_pec_values, std_true_values, std_noisy_values, std_pec_values

## Actual test call

In [ ]:
cnot_counts, oneq_counts, true_values, noisy_values, pec_values, avg_true_values, avg_noisy_values, avg_pec_values, std_true_values, std_noisy_values, std_pec_values = test_depths(depths, trials, n_qubits, shots, num_samples, circuit_type)

## Plotting

In [ ]:
plt.figure(figsize=(9, 5))
plot_depths = depths[:len(avg_true_values)]
kwargs = {"capsize": 5, "lw": 2}

plt.plot(plot_depths, avg_true_values, label="True", lw=2)

plt.errorbar(plot_depths, avg_noisy_values, yerr=std_noisy_values, label="Raw", **kwargs, fmt='o', ms=2.5)
plt.errorbar(plot_depths, avg_pec_values, yerr=std_pec_values, label="PEC", **kwargs, fmt='o', ms=2.5)

plt.xlabel("Depth")
plt.ylabel("Expectation value")
plt.title(f"""Real with {circuit_type} circuit using PEC on simulator using {n_qubits} qubits and {trials} trials per depth""")
plt.legend() 

save_data = True
import datetime
if save_data == True:

    dir_name = f"Real_PEC_{circuit_type}_nqubits_{n_qubits}_shots_{shots}_trials_{trials}_Test_Depths"
    time_key = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    try:
        os.mkdir(dir_name)
    except FileExistsError:
        pass
    plt.savefig(os.path.join(dir_name,f"Real_PEC_{circuit_type}_{n_qubits}"+ time_key +".pdf"))
    np.savetxt(os.path.join(dir_name, "cnot_counts" + time_key + ".txt"), cnot_counts)
    np.savetxt(os.path.join(dir_name, "oneq_counts" + time_key + ".txt"), oneq_counts)
    np.savetxt(os.path.join(dir_name, "true_values" + time_key + ".txt"), true_values)
    np.savetxt(os.path.join(dir_name, "noisy_values" + time_key + ".txt"), noisy_values)
    np.savetxt(os.path.join(dir_name, "pec_values" + time_key + ".txt"), pec_values)

    print("Data saved succesfully in", dir_name)

In [ ]:
# # Plot Errors
noisy_errors = np.abs([a_i - b_i for a_i, b_i in zip(avg_noisy_values, avg_true_values)])
pec_errors = np.abs([a_i - b_i for a_i, b_i in zip(avg_pec_values, avg_true_values)])

plt.figure(figsize=(9, 5))
plot_depths = depths[:len(avg_true_values)]
kwargs = {"capsize": 5, "lw": 2}

plt.errorbar(plot_depths, noisy_errors, yerr=std_noisy_values, label="Raw", **kwargs, fmt='o', ms=2.5)
plt.errorbar(plot_depths, pec_errors, yerr=std_pec_values, label="PEC", **kwargs, fmt='o', ms=2.5)

plt.xlabel("Depth")
plt.ylabel("Error in xpectation value")
plt.title(f"""Real with {circuit_type} circuit using PEC on simulator using {n_qubits} qubits and {trials} trials per depth""")
plt.legend() 

# save_data = True
import datetime
if save_data == True:

    dir_name = f"Real_PEC_{circuit_type}_nqubits_{n_qubits}_shots_{shots}_trials_{trials}_Test_Depths"
    time_key = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    try:
        os.mkdir(dir_name)
    except FileExistsError:
        pass
    plt.savefig(os.path.join(dir_name,f"Real_PEC_{circuit_type}_{n_qubits}_Errors"+ time_key +".pdf"))

    print("Data saved succesfully in", dir_name)